# Open Networks Ansible Guide
## oder: Kommen, Sehen, Planen, Automatisieren, Siegen

# Inhaltsverzeichnis

## Vorwort 
## Projekt Planung, Designentscheidungen
## Entwickeln, Testen, debbugen

# Vorwort

Dieses Notebook soll Leuten, die erste Erfahrungen mit Ansible gemacht haben, einen geführten Weg tiefer in die Materie darstellen. Er basiert auf Empfehlungen der allgemeinen Dokumentation und Erfahrungen aus Ansible anderen Automatisierungs und Softwareprojekten. 

## Hilfe zur Selbsthilfe
Effizient auf die Dokumentation zu zugreifen ist beim Erlernen, Verbessern und genrell Einsetzen von IT Skills entscheidend für die Effizienz... und die allgemeine Geistige Gesundheit.

Der folgende Ablauf hat sich bei uns beim Nachschlagen bewährt:

- Tutorials, Best practices, Verbesserung von skills:
  - Die links unter https://docs.ansible.com/ sind alle super
  - Kurse unter redhat

- Bei Suche nach dem passenden Modul: 
    - ansible-doc -l | grep && ansible-doc 
    - autcomplete in VScode
    - suche auf https://docs.ansible.com/ansible/latest/
    - www.google.com
    
- 




In [1]:
!ansible-doc debug

> DEBUG    (/home/fkain/.local/share/virtualenvs/ansible-jupyter-w17giIN2/lib/python3.7/site-packages/ansible/modules/utilities/logic/debug.py)

        This module prints statements during execution and can be
        useful for debugging variables or expressions without
        necessarily halting the playbook. Useful for debugging
        together with the 'when:' directive. This module is also
        supported for Windows targets.

  * note: This module has a corresponding action plugin.

OPTIONS (= is mandatory):

- msg
        The customized message that is printed. If omitted, prints a
        generic message.
        [Default: Hello world!]

- var
        A variable name to debug.  Mutually exclusive with the 'msg'
        option.
        [Default: (null)]

- verbosity
        A number that controls when the debug is run, if you set to 3
        it will only run debug when -vvv or above
        [Default: 0]
        version_added: 2.1


NOTES:
      * This module is also suppor

<img src="media/vscode_autocomplete.png" width="100%" />

## Indikatoren für Planung und Design
| Indikator  | Bedeutung            |
| ---------- | -------------------- |
| Projekt wird wachsen   | Standard Projekt Layout verwenden   |
| mehr als eine Person arbeitet daran | Versionskontrolle! |
| mehr als eine Person könnte es benutzen | ansible tower evaluieren |
| Aufgaben lassen sich wieder verwerten | in Rollen zusammenfassen , ansible-galaxy verwenden|


1. TODO: Pest practices Directory struktur
2. TODO: Entscheidungstabelle: Wann welches element
- Inventory yml oder ini?
- Inventory file oder folder?
- Group vars oder folder?
- Wann play
- Idempotenz. Vorteile Grenzen und Lösungen

# Projekt Folder Layout
<img src="media/dir_layout.png" width="100%" />

## Alternatives Inventory Layout
<img src="media/alt_inventory_layout.png" width="100%" />

### Notizen
Für kleine Projekte wirken die Layouts aufgeblasen, bei größeren sind sie nötig.

Gründe für die Verwendung des Layouts trotz anfänglich höheren Lernaufwandes:
- Konsistenz über Projekte hinweg
- Modularität erleichtert Hinzufügen modifizieren von Funktionen über neue Rollen
- Einbinden von ansible-galaxy Rollen
- Steuerelemente (Variablen) sind durch Verzeichnisstruktur bereits vorgegeben

# Entwickeln, Testen, debugen

## Variablen Hierachie (Precedence)
Bestimmt durch ihre Herkunft überschreiben unten gelistete Variablentypen die oberen

1. command line values (eg “-u user”)
1. role defaults [1]
1. group_vars/all [3]
1. group_vars/* [3]
1. host_vars [3]
1. play vars
1. role vars (defined in role/vars/main.yml)
1. task vars (only for the task)
1. include_vars (load variables explicitly from a file)
1. set_facts / registered vars (explicitly set host fact)
1. role params (set in the playbook)
1. include params ( parsed with the include statement
1. extra vars (parsed on command line with -e 'var_xy=jaguar var_z=present')

## Vollständige Variablen Hierachie (Precedence)
1. command line values (eg “-u user”)
1. role defaults [1]
1. inventory file or script group vars [2]
1. inventory group_vars/all [3]
1. playbook group_vars/all [3]
1. inventory group_vars/* [3]
1. playbook group_vars/* [3]
1. inventory file or script host vars [2]
1. inventory host_vars/* [3]
1. playbook host_vars/* [3]
1. host facts / cached set_facts [4]
1. play vars
1. play vars_prompt
1. play vars_files
1. role vars (defined in role/vars/main.yml)
1. block vars (only for tasks in block)
1. task vars (only for the task)
1. include_vars
1. set_facts / registered vars
1. role (and include_role) params
1. include params
1. extra vars (always win precedence)

from: https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#variable-precedence-where-should-i-put-a-variable

### Beispiel include params

In [2]:
#inventory
[all]
localhost

In [2]:
#play
name: Over write example
hosts: all
vars: 
    vsphere_user: OLD


TASK [Gathering Facts] *********************************************************
ok: [localhost] => {
    "_ansible_verbose_override": true,
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "10.60.147.88"
        ],
        "ansible_all_ipv6_addresses": [],
        "ansible_apparmor": {
            "status": "enabled"
        },
        "ansible_architecture": "x86_64",
        "ansible_bios_date": "02/15/2019",
        "ansible_bios_version": "N20ET41W (1.26 )",
        "ansible_cmdline": {
            "BOOT_IMAGE": "/boot/vmlinuz-4.18.0-18-generic",
            "quiet": true,
            "ro": true,
            "root": "UUID=86012471-3357-4ea8-bcfa-94b35df22d09",
            "splash": true,
            "vt.handoff": "1"
        },
        "ansible_date_time": {
            "date": "2019-05-07",
            "day": "07",
            "epoch": "1557234214",
            "hour": "15",
            "iso8601": "2019-05-07T13:03:34Z",
            "iso8601_basic": "20

In [3]:
debug:
    var: vsphere_user

TASK [debug] *******************************************************************
ok: [localhost] => {
    "vsphere_user": "OLD"
}


In [4]:
include_tasks: included_task.yml
vars:
    vsphere_user: vwmareNEW

fatal: [localhost]: FAILED! => A malformed block was encountered while loading a block
TASK [Gathering Facts] *********************************************************
ok: [localhost] => {
    "_ansible_verbose_override": true,
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "10.60.147.88"
        ],
        "ansible_all_ipv6_addresses": [],
        "ansible_apparmor": {
            "status": "enabled"
        },
        "ansible_architecture": "x86_64",
        "ansible_bios_date": "02/15/2019",
        "ansible_bios_version": "N20ET41W (1.26 )",
        "ansible_cmdline": {
            "BOOT_IMAGE": "/boot/vmlinuz-4.18.0-18-generic",
            "quiet": true,
            "ro": true,
            "root": "UUID=86012471-3357-4ea8-bcfa-94b35df22d09",
            "splash": true,
            "vt.handoff": "1"
        },
        "ansible_date_time": {
            "date": "2019-05-07",
            "day": "07",
            "epoch": "1557234229",
            "hour":


Playbook ended
Context lost!


### Vergleiches-Operatoren"
| Operator  | Bedeutung            |
| --------- | -------------------- |
| `<`       | kleiner als          |
| `<=`      | kleiner oder gleich  |

In [ ]:
adasd

# Entwickeln, Testen, debbugen

## Entwicklungsprozess mit Ansible
## Tools und Hacks
## Was tun wenn schmutzig wird


## Tools und Hacks

### ansible-lint
- https://docs.ansible.com/ansible-lint/
- https://github.com/ansible/ansible-lint/tree/master/examples
<img src="media/ansible-lint-play.png"/>
<img src="media/ansible-lint.png"/>


### ansible-jupyter  
https://github.com/ansible/ansible-jupyter-kernel/tree/master/notebooks
Interaktive Gestaltung und Demonstration von Playbooks durch python jupyter notebooks.

Diese Präsentation bassiert darauf und kann von 


In [4]:
!ansible-lint include_tasks_play.yml


 [WARNING]: module include_tasks_play.yml not found in: /home/fkain/.ansible/pl
ugins/modules:/usr/share/ansible/plugins/modules:/home/fkain/.local/share/virtu
alenvs/ansible-jupyter-w17giIN2/lib/python3.7/site-packages/ansible/modules


# good to know
- variablen hierachie (Precedence)
- tags
- debugging tools (tags, skip start with)

